------------------------------------------------------------------------------------------------------------------------------------------------------

# SCINT (for begginers)

This is a python library to process and analyse raw data from the IR02 lab.

Access the [DOCUMENTATION](https://cython-tools.readthedocs.io/en/latest/) for more information.

You have cloned the repository into a local directory and you are ready to start. 

### PLEASE! When you are done with the tutorial, move to the ```notebooks``` folder to start your own analysis 

In this way, you will be able to keep the tutorial as a reference for future use (or others to use it).

------------------------------------------------------------------------------------------------------------------------------------------------------

## STEP 0: prepare the environment
    
Open a terminal and run to prepare the packages needed for the macros:
```bash
cd scripts
sh setup.sh 
```
Alternatevely, you can open /scripts/requirements.txt and install them manually.

For working with real data you have two options:
1) Download the data:
    ```bash
    sh copy_data.sh AFS_USER pcaeXYZ
    ```
2) Mount in an empty data folder the ```TUTORIAL``` one as:
    ```bash
    sshfs AFS_USER@pcaeXYZ.ciemat.es:/pc/choozdsk01/palomare/SCINT/ ../data
    ```


For both cases you will find template data in the ```data/TUTORIAL/raw``` folder which will be used to run the tutorial.

In ```input/TUTORIAL.txt``` you have an example of the input file needed to run the macros with all the variables,runes and paths needed.

## STEP 1: understand the library

Once you have setup the environment, you can start to play with the library. You can see different folders:
```bash
00TUTORIAL -> this folder contains a tutorial notebook (this one) and template notebooks you can use as reference
fit_data   -> in this folder fits results will be stored
input      -> in this folder you will find the input files needed to run the macros
lib        -> in this folder you will find the all the functions of the library itself
macros     -> in this folder you will find the macros to run the analysis. The are sorted by order of execution
notebooks  -> in this folder you will find the notebooks to run the analysis, PREPARED TO BE CHANGED AND NOT UPDATED IN THE REPOSITORY
```

Everytime you run a macro you can choose two options: 
1) Run it in the `guide` mode and the macro will ask you for the input variables needed (see an example for the input file)

<img src="images/starting_macro.png" alt="start"/>

2) Once you are sure about the variables to use you can run the macro with the defined flags:

```python
-h or --help
-i or --input_file
-l or --load_preset
-s or --save_preset
-k or --key
-v or --variables
-r or --runs
-c or --channels
-f or --filter
-d or --debug
```

### **Time to start the tutorial!**

## STEP 2: process the raw data


```bash
cd macros
python3 00Raw2Np.py        (-i TUTORIAL -d y)
```
```
<img src="images/00Raw2Np.png" alt="Raw2Np"/>


```bash
python3 01PreProcess.py     (-i TUTORIAL -k RawADC -d y)
```
<img src="images/01PreProcess.png" alt="PreProcess"/>

```bash
python3 02AnaProcess.py     (-i TUTORIAL -k AnaADC -d y)
```
<img src="images/02Process.png" alt="Process"/>

```bash
python3 03Integration.py     (-i TUTORIAL -k RawADC -d y)
```

<img src="images/03Integration.png" alt="Integration"/>

## STEP 2: visualize data

In [2]:
# The first line in every macro should be the following for importing all the packages
import sys; sys.path.insert(0, '../'); from lib import *;
figure_features()

You don't have latex installed. Changing default configuration to tex=Foptionalalse


In [3]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,                # Time in microseconds (True/False)
    "NORM":        False,               # Runs can be displayed normalised (True/False)
    "ALIGN":       True,                # Aligns waveforms in peaktime (True/False)
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",                  # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,               # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",    # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,               # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "SHOW":        True
    }

In [5]:
INPUT_FILE = "Jan22"; OV = 1; PRESET ="EVA"
info = read_input_file(INPUT_FILE)  # Read input file
channels = [0,6]

#-------------------------------- LOAD RUNS ---------------------------------#
run_keys = ["CALIB_RUNS","LIGHT_RUNS","ALPHA_RUNS","MUONS_RUNS","NOISE_RUNS"]
nruns = dict.fromkeys(run_keys)
for key in run_keys:
    try:               nruns[key] = info[key][OV] # Store runs in dictionary
    except IndexError: nruns.pop(key)
print(nruns)

runs = dict.fromkeys(nruns.keys())
for run in runs: runs[run] = load_npy(np.asarray([nruns[run]]).astype(int),np.asarray(channels).astype(int),preset=PRESET,info=info,compressed=True)
#----------------------------------------------------------------------------#

time = np.arange(len(runs[list(nruns.keys())[0]][runs[list(nruns.keys())[0]]["NRun"][0]][channels[0]]["AnaAveWvf"][0]))*4e-9

{'CALIB_RUNS': 2, 'LIGHT_RUNS': 58, 'ALPHA_RUNS': 13, 'MUONS_RUNS': 47}
load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!

load_npy --> DONE!



## INDIVIDUAL EVENTS

By running the next function you can dynamically visualize the RAW events of the selected run to see everything is OK.

(In future versions we will make possible to visualize any kind of event with the same function.)



```bash
%run run ../macros/0UVisEvent.py
```

And then browse your ```waveW.dat``` file (or the preprocessed one ```RawADC.npz```) to see the events. You can go to the next one or the previus one with the arrows.

In [6]:
%run ../macros/0UVisEvent.py

Check also ``EventDisplay.ipynb``

## NOISE 

In [7]:
noise = runs["NOISE_RUNS"][runs["NOISE_RUNS"]["NRun"][0]][channels[0]]["AnaAveWvf"][0]
sp    = np.fft.rfft(noise)
freq  = np.fft.rfftfreq(len(time), d = 4e-9)

fig = px.line(x=freq, y=sp.imag)
custom_plotly_layout(fig, title='FFT', xaxis_title='Frequency [Hz]', yaxis_title='FFT').show()


Check also ``Noise.ipynb`` and ``PeakPedVariables.ipynb``

## LASER

In [8]:
LASER = []
for c,ch in enumerate(channels):
    laser = runs["LIGHT_RUNS"][runs["LIGHT_RUNS"]["NRun"][0]][channels[c]]["AnaAveWvf"][0]
    laser = laser / np.max(laser)
    
    ref_max_idx = np.argmax(laser)
    idx_move    = np.argmax(laser)
    laser       = shift(laser, ref_max_idx-idx_move, cval = 0)

    LASER.append(laser)
    
df_laser = pd.DataFrame({"Time":time,"Laser_ch0":LASER[0],"Laser_ch6":LASER[1]})

In [9]:
fig_laser = px.line(df_laser, x="Time", y=["Laser_ch0","Laser_ch6"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_laser.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_laser,["Laser (SiPM)","Laser (XA)"])
custom_plotly_layout(fig_laser,"Time","Normalized ADC","Laser signals").show()

## ALPHA

In [10]:
ALPHA = []
for c,ch in enumerate(channels):
    alpha = runs["ALPHA_RUNS"][runs["ALPHA_RUNS"]["NRun"][0]][channels[c]]["AnaAveWvf"][0]
    alpha = alpha / np.max(alpha)

    ref_max_idx = np.argmax(alpha)
    idx_move    = np.argmax(alpha)
    alpha       = shift(alpha, ref_max_idx-idx_move, cval = 0)
    ALPHA.append(alpha)


df_alpha = pd.DataFrame({"Time":time,"Alpha_ch0":ALPHA[0],"Alpha_ch6":ALPHA[1]})

In [11]:
fig_alpha = px.line(df_alpha, x="Time", y=["Alpha_ch0","Alpha_ch6"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_alpha.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_alpha,["Alpha (SiPM)", "Alpha (XA)"])
custom_plotly_layout(fig_alpha,"Time","Normalized ADC","Alpha signals").show()

Check also ``AverageWvfs.ipynb``

## PedSTD

In [12]:
alpha_runs = runs["ALPHA_RUNS"][runs["ALPHA_RUNS"]["NRun"]]
my_runs_alpha = load_npy(np.asarray(alpha_runs).astype(int),np.asarray(channels).astype(int),preset="ALL",info=info,compressed=True)
cut_dict = {'cut_df': [True,[[["0"],"AnaPeakTime","bigger_than",700,False]]], 'cut_lin_rel': [False,[]], 'cut_peak_finder': [False,[]]}
label, my_runs_alpha = cut_selector(my_runs_alpha, {'runs': ['25'], 'input_file': ['TUTORIAL'], 'filter':cut_dict,'debug': True, 'channels': ['0']}, debug=True)
average_wvfs(my_runs_alpha, key="RawADC", label="Ana",cut_label="Pedestal", centering="NONE", debug=True)

print(my_runs_alpha[alpha_runs[0]][channels[0]].keys())
AlphaNoise = my_runs_alpha[alpha_runs[0]][channels[0]]["AnaAveWvfPedestal"][0]
df_alphanoise = pd.DataFrame({"Time":time,"AlphaNoise":AlphaNoise})

load_npy --> DONE!

load_npy --> DONE!

---- LET'S CUT! ----
... Cutting events for run 25 channel 0 with AnaPeakTime bigger_than 700.00 ...
Nº of selected events from total events: 9890 (100.00%)
-> Found label Raw form key RawADC
Calculating average wvf with cuts
Computing ANA wvfs from RAW
Averaging wvf: AnaAveWvfPedestal
-> Found label Raw form key RawADC
Calculating average wvf with cuts
Computing ANA wvfs from RAW
Averaging wvf: AnaAveWvfPedestal
Average waveform calculated
dict_keys(['AnaPreTriggerMax', 'RawPedEnd', 'RawPedMax', 'AnaValleyAmp', 'RawValleyTime', 'AnaAveWvf', 'AnaPreTriggerMean', 'PChannel', 'AnaPreTriggerSTD', 'AnaPChannel', 'Label', 'AnaPedSTD', 'RawPChannel', 'AnaChargePedRange1', 'AnaPedMin', 'RawValleyAmp', 'AnaPeakAmp', 'AnaPeakTime', 'AnaChargePedRange2', 'AnaChargeRangeDict', 'RawPeakTime', 'AnaPedLim', 'AnaValleyTime', 'TimeStamp', 'RawPreTriggerMax', 'RawPeakAmp', 'RawPedStart', 'AnaPedStart', 'RawADC', 'AnaChargeAveRange', 'NBinsWvf', 'AnaPreTriggerMin'

In [13]:
fig_alphanoise = px.line(df_alphanoise, x="Time", y=["AlphaNoise"], height=700, width = 900, line_shape = "hv", log_y = False, template="presentation")
fig_alphanoise.update_traces(line_width=2) # Modifica el grosor de las líneas
custom_legend_name(fig_alphanoise,["AlphaNoise (SiPM)"])
custom_plotly_layout(fig_alphanoise,"","Normalized ADC","Alpha Noise").show()